In [3]:
from vectorbtpro import *
import os

### Data Ingest 

In [4]:
'''
polygon_api_key = os.getenv('POLYGON_API_KEY')
vbt.PolygonData.set_custom_settings(
    client_config=dict(
        api_key=polygon_api_key
    )
)
data = vbt.PolygonData.pull(
    ["X:BTCUSD",
    "X:ETHUSD",
    "X:SOLUSD"],
    start="2019-01-01",
    end="2024-11-01",
    timeframe="1 day"
)

data.to_hdf('priceseries.h5')
'''

data = vbt.HDFData.pull(['priceseries.h5/X:BTCUSD'])
#data = vbt.HDFData.pull('priceseries.h5')


performance_metrics = [
    'max_dd', 
    'max_dd_duration',
    'sortino_ratio',
    'omega_ratio',
    'profit_factor',
    'expectancy',
    'total_orders',
    'win_rate',
    'avg_winning_trade_duration',
    'avg_losing_trade_duration'
]

open_price = data.get('Open')
close_price = data.get('Close')
high_price = data.get('High')
low_price = data.get('Low')

### Indicator Parameter Space

In [5]:
wtypes = ["simple", "exp", "wilder"]
rsi_win = list(range(8, 21))
bbands_win = np.arange(10, 50, 5)
alphas = np.arange(1.5, 3.0, 0.5)

'\nrsi_lower_ths = list(range(20, 31))\nrsi_upper_ths = list(range(70, 81))\nrsi_lower_ths_prod, rsi_upper_ths_prod = zip(*product(rsi_lower_ths, rsi_upper_ths))\n'

Bollinger-Band Calculation: $\alpha$ = number of standard deviations
$$
\begin{align*}
\text{Middle Band} &= \text{SMA}(\text{close}, n) \\
\text{Upper Band} &= \text{SMA}(\text{close}, n) + \alpha \times \sigma(\text{close}, n) \\
\text{Lower Band} &= \text{SMA}(\text{close}, n) - \alpha \times \sigma(\text{close}, n) \\\\
\text{Bandwidth} &= \frac{\text{Upper Band} - \text{Lower Band}}{\text{Middle Band}}
\end{align*}
$$

In [6]:
bbands = vbt.BBANDS.run(
    close_price,
    window=bbands_win,
    wtype=wtypes,
    alpha=alphas,
    param_product=True
)

### Trade Conditions

#### Bollinger Band Signal
<u>Sell signal</u>: Downward breakout while expanding  
If the daily low is below the lower band *AND* the bandwidth is above the lower bandwidth threshold.

<u>Buy signal</u>: Upward breakout while squeezing  
If the daily high is above the upper band *AND* the bandwidth is below the upper bandwidth threshold.

In [7]:
bandwidth = (bbands.upper - bbands.lower) / bbands.middle

entry_cond1 = data.get("Low").vbt < bbands.lower
entry_cond2 = bandwidth > 0.3
entry_cond3 = data.get("High").vbt > bbands.upper
entry_cond4 = bandwidth < 0.15

in_cond1, in_cond2, in_cond3, in_cond4 = vbt.pd_acc.x(entry_cond1, entry_cond2, entry_cond3, entry_cond4)
entry_signals = (in_cond1.vbt & in_cond2).vbt | (in_cond3.vbt & in_cond4)

exit_cond1 = data.get("High").vbt > bbands.upper
exit_cond2 = bandwidth > 0.3
exit_cond3 = data.get("Low").vbt < bbands.lower
exit_cond4 = bandwidth < 0.15

out_cond1, out_cond2, out_cond3, out_cond4 = vbt.pd_acc.x(exit_cond1, exit_cond2, exit_cond3, exit_cond4)
exit_signals = (out_cond1.vbt & out_cond2).vbt | (out_cond3.vbt & out_cond4)

# select only the first signal (entry or exit)
exits = exit_signals.vbt.signals.first_after(entry_signals, reset_wait=0)
entries = entry_signals.vbt.signals.first_after(exit_signals)


How many Trades are placed?

In [11]:
trades = dict({"entries":entries.sum(), "exits":exits.sum()})
print(trades)

{'entries': bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.5       X:BTCUSD    131
                     2.0       X:BTCUSD     84
                     2.5       X:BTCUSD     56
           exp       1.5       X:BTCUSD     56
                     2.0       X:BTCUSD     15
                                          ... 
45         exp       2.0       X:BTCUSD      6
                     2.5       X:BTCUSD      3
           wilder    1.5       X:BTCUSD      4
                     2.0       X:BTCUSD      4
                     2.5       X:BTCUSD      3
Length: 72, dtype: int64, 'exits': bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.5       X:BTCUSD    143
                     2.0       X:BTCUSD     86
                     2.5       X:BTCUSD     57
           exp       1.5       X:BTCUSD     56
                     2.0       X:BTCUSD     15
                                          ... 
45         exp       2.0       X:BTCUSD      6
                     2.5   

### Create a Portfolio from Entry and Exit Conditions

In [12]:
pf = vbt.Portfolio.from_signals(
    close=close_price, 
    entries=entries, 
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)
stats_df = pf.stats(performance_metrics, agg_func=None)

Determine the omega-optimized portfolio

In [14]:
stats_df.sort_values(by="Omega Ratio", ascending=False)
stats_df.head()

Max Drawdown [%] Max Drawdown Duration  \
bb_window bb_wtype bb_alpha symbol                                             
10        simple   1.5      X:BTCUSD         21.397939             1096 days   
                   2.0      X:BTCUSD         30.048381             1329 days   
                   2.5      X:BTCUSD         46.968356             1298 days   
          exp      1.5      X:BTCUSD         25.316769             1190 days   
                   2.0      X:BTCUSD         28.776171             1298 days   

                                      Sortino Ratio  Omega Ratio  \
bb_window bb_wtype bb_alpha symbol                                 
10        simple   1.5      X:BTCUSD       2.378246     1.391712   
                   2.0      X:BTCUSD       1.800596     1.288191   
                   2.5      X:BTCUSD       1.066056     1.155533   
          exp      1.5      X:BTCUSD       1.845574     1.294363   
                   2.0      X:BTCUSD       2.707402     1.518401   

                                      Profit Factor  Expectancy  Total Orders  \
bb_window bb_wtype bb_alpha symbol                                              
10        simple   1.5      X:BTCUSD       1.994903    2.794926           231   
                   2.0      X:BTCUSD       1.844302    2.596688           159   
                   2.5      X:BTCUSD       1.444180    2.368029           103   
          exp      1.5      X:BTCUSD       2.189260    4.437632           103   
                   2.0      X:BTCUSD      10.657807   25.327916            27   

                                      Win Rate [%] Avg Winning Trade Duration  \
bb_window bb_wtype bb_alpha symbol                                              
10        simple   1.5      X:BTCUSD     36.521739 15 days 14:17:08.571428571   
                   2.0      X:BTCUSD     39.240506 18 days 10:50:19.354838709   
                   2.5      X:BTCUSD     47.058824           23 days 04:00:00   
          exp      1.5      X:BTCUSD     37.254902 32 days 08:50:31.578947368   
                   2.0      X:BTCUSD     69.230769           61 days 10:40:00   

                                     Avg Losing Trade Duration  
bb_window bb_wtype bb_alpha symbol                              
10        simple   1.5      X:BTCUSD 5 days 14:47:40.273972602  
                   2.0      X:BTCUSD           8 days 23:00:00  
                   2.5      X:BTCUSD          22 days 11:33:20  
          exp      1.5      X:BTCUSD          12 days 15:45:00  
                   2.0      X:BTCUSD          63 days 12:00:00

In [17]:
pf[(10, "simple", 1.5)].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '749bf763-566c-438e-98df-f438dad86773',
              'x': array([datetime.datetime(2019, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2019, 1, 3, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 10, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 30, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 31, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([110.65010351, 110.65010351, 110.65010351, ..., 110.65010351,
